# Example Code for Estimating the Ground State Energy of Hydroxyl (·OH)

## Basic Installation

Install required package, we highly recommend participant to use qiskit platform, or at least participants can finish preprocessing at other platform and transfer the circuit to qiskit format, since our noise model is from IBM real machine backend and we restricted some algorithmic seeds which could be varied from different platform.

In [ ]:
!pip install qiskit
!pip install qiskit-nature[pyscf] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 10.1 MB/s eta 0:00:00


In [ ]:
!pip install qiskit_aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.8 MB/s eta 0:00:00


In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper,ParityMapper,QubitConverter
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
import numpy as np
import pylab
import qiskit.providers
from qiskit import Aer,pulse, QuantumCircuit
from qiskit.utils import QuantumInstance, algorithm_globals
import time

Here we require paticipants to fix the algorithm seed in qiskit. *MUST* translate other format circuit to qiskit before any place need algorithm seed. And we give 20, 21, 30, 33, 36, 42, 43, 55, 67, 170 as seeds that requires to run, and the result will be calculated as the average of results from each seed. And please use shots as 4000.

In [ ]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
iterations = 125
shot = 4000

## Generate Hamiltonian and Pauli String

At this step, the example code uses PySCF to generate the hamiltonian of hydroxyl with basis function as 'sto3g' to fit the spin orbital, then uses JordanWignerMapper to map the fermionic terms to pauli strings. To be noticed, other chemistry tool also allowed to be used at this step, but keep in mind to use 'sto-3g' and Jordan Wigner Mapper which should gives 12 qubits and 631 paulil terms.

In [ ]:
ultra_simplified_ala_string = """
O 0.0 0.0 0.0
H 0.45 -0.1525 -0.8454
"""

driver = PySCFDriver(
    atom=ultra_simplified_ala_string.strip(),
    basis='sto3g',
    charge=1,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)
qmolecule = driver.run()

In [ ]:
hamiltonian = qmolecule.hamiltonian
coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)
second_q_op = hamiltonian.second_q_op()

Polynomial Tensor
 "+-":
[[-3.21461222e+01  5.59899100e-01  1.87617178e-01 -8.82935672e-16
   5.24041109e-16 -1.94702445e-01]
 [ 5.59899100e-01 -7.35898345e+00 -2.46352634e-01  1.62827403e-15
  -6.24537640e-16  9.51226718e-01]
 [ 1.87617178e-01 -2.46352634e-01 -6.56995119e+00 -2.60483168e-15
   3.30435485e-15 -1.09726793e+00]
 [-8.81884087e-16  1.65690925e-15 -2.62384089e-15 -6.94886145e+00
  -5.91762610e-16  6.31170002e-15]
 [ 5.24315261e-16 -8.02073589e-16  2.84157066e-15 -6.38131808e-16
  -6.94886145e+00 -2.17814296e-15]
 [-1.94702445e-01  9.51226718e-01 -1.09726793e+00  6.25309946e-15
  -1.88022334e-15 -4.64967973e+00]]
 "++--":
[[[[ 4.74977044e+00 -4.38465691e-01 -1.51436760e-01  7.24476990e-16
    -3.88796601e-16  1.59790984e-01]
   [-4.38465691e-01  6.47204045e-02  1.84429506e-02 -9.30841809e-17
     5.68926263e-17 -2.66865302e-02]
   [-1.51436760e-01  1.84429506e-02  2.46189939e-02  1.21320840e-17
    -1.54173925e-17  6.40512562e-03]
   [ 7.24430816e-16 -9.31221507e-17  1.19056

In [ ]:
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
qubit_op = converter.convert(second_q_op)

We recommend to use classical minimum eigensolver to obtain a reference energy at this step. In case some of the classical minimum eigensolver donot directly gives nuclear repulsion energy, we give reference energies below: *Comupted Energy*: -78.75252123, *Nuclear Repulsion_energy*: 4.36537496654537. *Obtained Reference Ground State Energy*: -74.38714627.

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [ ]:
result = solver.solve(qmolecule)
print(result.computed_energies)

[-78.75252123]


In [ ]:
print(result.nuclear_repulsion_energy)

4.36537496654537


In [ ]:
ref_value = result.computed_energies + result.nuclear_repulsion_energy
print(ref_value)

[-74.38714627]


## Construct Ansatz

At this stage, you can implement various techniques to search good ansatz architecture which is important for variational quantum algorihms. Moreover, how to obtain a good initial state is a good topic to do research, we require participant to self-reflection there techniques (include the techniques for preprocessing ansatz or initial states) with maximum 10 points, and submit a short description for used techniques, we will have three graders to evaluate the techniques.

In [ ]:
ansatz = UCCSD(
    qmolecule.num_spatial_orbitals,
    qmolecule.num_particles,
    mapper,
    initial_state=HartreeFock(
        qmolecule.num_spatial_orbitals,
        qmolecule.num_particles,
        mapper,
    ),
)


In [ ]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

In [ ]:
estimator = Estimator(
    backend_options = {
        'method': 'statevector',
        'device': 'CPU'
        # 'noise_model': noise_model
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler':seed_transpiler
    }
)

In [ ]:
vqe_solver = VQE(estimator, ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [ ]:
start_time = time.time()
calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(qmolecule)
end_time = time.time()
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -57.121081083643
  - computed part:      -57.121081083643
~ Nuclear repulsion energy (Hartree): 4.365374966545
> Total ground state energy (Hartree): -52.755706117097
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 8.008 S: 0.334 S^2: 0.445 M: -0.005
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.85037676  -0.28818323  -1.59757447]
 
  0: 
  * Electronic dipole moment (a.u.): [1.104033287945  -0.383250343741  -2.067354598683]
    - computed part:      [1.104033287945  -0.383250343741  -2.067354598683]
  > Dipole moment (a.u.): [-0.253656527945  0.095067113741  0.469780128683]  Total: 0.542284758765
                 (debye): [-0.644730523886  0.24163647805  1.194061871504]  Total: 1.378350241751
 


## Calculate the Accuracy (Most Important Metric)

In [ ]:
result = res.computed_energies + res.nuclear_repulsion_energy
error_rate = abs(abs(ref_value - result) / ref_value * 100)
print("Error rate: %f%%" % (error_rate))

Error rate: 29.079540%


## Obtain the Duration of Quantum Circuit

In [ ]:
from qiskit.providers.fake_provider import *
backend = FakeMontreal()

In [ ]:
with pulse.build(backend) as my_program1:
  pulse.call(ansatz)

In [ ]:
my_program1.duration

20048096